In [ ]:
# 1. Install YOLOv8
!pip install ultralytics -q

In [ ]:
# 2. Mount Google Drive (untuk save model)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 3. Upload dataset.zip
# Upload file zip yang berisi folder: train/, val/, test/, data.yaml
from google.colab import files
print("Upload file dataset.zip...")
uploaded = files.upload()

In [ ]:
# 4. Extract dataset
!unzip -q dataset.zip -d /content/
!ls /content/data/

In [ ]:
# 5. Buat data.yaml yang sesuai path Colab
yaml_content = """path: /content/data
train: train/images
val: val/images
test: test/images

names:
  0: head_down
  1: head_up
  2: head_left
  3: head_right
  4: head_center
  5: eye_center
  6: eye_left
  7: eye_right
  8: eye_up
  9: eye_down
  10: eye_closed
  11: lip_closed
  12: lip_open

nc: 13
"""

with open('/content/data/data.yaml', 'w') as f:
    f.write(yaml_content)

print("data.yaml created!")
!cat /content/data/data.yaml

In [ ]:
# 6. Cek GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# 7. Training YOLOv8
from ultralytics import YOLO

# Load pretrained model
model = YOLO('yolov8n.pt')

# Training
results = model.train(
    data='/content/data/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    project='/content/runs',
    name='cheating_detector',
    exist_ok=True,
    pretrained=True,
    optimizer='Adam',
    lr0=0.001,
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
)

In [ ]:
# 8. Lihat hasil training
from IPython.display import Image, display

# Confusion matrix
display(Image('/content/runs/cheating_detector/confusion_matrix.png', width=600))

# Results
display(Image('/content/runs/cheating_detector/results.png', width=800))

In [ ]:
# 9. Test model
model = YOLO('/content/runs/cheating_detector/weights/best.pt')

# Validasi
metrics = model.val()
print(f"\nmAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

In [ ]:
# 10. Copy model ke Google Drive
!mkdir -p /content/drive/MyDrive/CheatingDetector
!cp /content/runs/cheating_detector/weights/best.pt /content/drive/MyDrive/CheatingDetector/
!cp /content/runs/cheating_detector/weights/last.pt /content/drive/MyDrive/CheatingDetector/
print("\n✅ Model saved to Google Drive: /CheatingDetector/best.pt")

In [ ]:
# 11. Download model langsung
from google.colab import files
files.download('/content/runs/cheating_detector/weights/best.pt')